In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("..")
from plot_OpSims import get_metric_medians
from script_utils import get_opsim_areas, get_wfd_mask

In [3]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [4]:
# import convenience functions
import sys 
sys.path.append("../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

## Load the results
We first load the results of the run into memory. We will read jointly the FBS v1.5 and FBS v1.6 results, and will differentiate at the time of plotting.

In [5]:
NSIDE=64
your_username = "rjassef"
folder_mafoutput = "Color_EM5_{0:d}".format(NSIDE)
resultDbPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [6]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [7]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [8]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [9]:
# check keys
bd_keys = list(bundleDicts[dbRuns[15]].keys())
print(bd_keys, len(list(bundleDicts.keys())), len(list(bundleDicts_raw.keys())))
print(Keys)

[(1, 'Exgalm5_color_with_cuts_AGN_ug_mean'), (2, 'Exgalm5_color_with_cuts_AGN_gr_mean'), (3, 'Exgalm5_color_with_cuts_AGN_ri_mean'), (4, 'Exgalm5_color_with_cuts_AGN_iz_mean'), (5, 'Exgalm5_color_with_cuts_AGN_zy_mean')] 190 190
[(1, 'Exgalm5_color_with_cuts_AGN_ug_mean'), (2, 'Exgalm5_color_with_cuts_AGN_gr_mean'), (3, 'Exgalm5_color_with_cuts_AGN_ri_mean'), (4, 'Exgalm5_color_with_cuts_AGN_iz_mean'), (5, 'Exgalm5_color_with_cuts_AGN_zy_mean')]


## WFD Filter

Load the WFD footprint as a mask, and add it to the mask of each run.

In [10]:
WFDfp_folder_mafoutput = "WFDfootprint_{0:d}".format(NSIDE)
WFDfp_resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, WFDfp_folder_mafoutput)
WFDfp_metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, WFDfp_folder_mafoutput)

In [11]:
wfd_mask = get_wfd_mask(WFDfp_folder_mafoutput, WFDfp_resultDbPath, WFDfp_metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [12]:
for run in dbRuns:
    for key in bundleDicts[run].keys():
        mask = bundleDicts[run][key].metricValues.mask
        bundleDicts[run][key].metricValues.mask = mask | wfd_mask[run]

## Process the metric results for each OpSim and save the results in a table.

Note that in general, the rank will be from worst (0%) to best (100%).

We will base the results on Matthew Temples i=24.5, E(B-V)=0 quasar color models, and we will output them at z=1, 2 and 3, which are covered by all filters.

In [13]:
from quasar_colors.quasar_colors import Temple_colors
zs = np.array([1., 2., 3.])
imag = 24.5

In [14]:
mds = dict()
filters = ["u","g","r","i","z","y"]
for j in range(len(filters[:-1])):
    color_name = "{0}-{1}".format(filters[j], filters[j+1])
    quasar_colors = Temple_colors(color_name,zs,imag)
    Keyname = 'Exgalm5_color_with_cuts_AGN_{0}{1}_mean'.format(filters[j], filters[j+1])
    for Key_aux in Keys:
        if Key_aux[1]==Keyname:
            Key = Key_aux
            break
    mds_aux = np.array(get_metric_medians(Key, bundleDicts))
    mds[color_name] = np.tile(mds_aux, (len(zs),1)).T - quasar_colors

In [15]:
mds_rank = dict()
for j in range(len(filters[:-1])):
    color_name = "{0}-{1}".format(filters[j], filters[j+1])
    mds_argsort = np.argsort(np.abs(mds[color_name]), axis=0)
    mds_rank[color_name] = np.zeros(mds_argsort.shape)
    for k in range(mds_rank[color_name].shape[0]):
        for i in range(mds_rank[color_name].shape[1]):
            rank_percent = (np.argwhere(k==mds_argsort[:,i])[0,0]+1)/mds_argsort.shape[0] * 100.
            mds_rank[color_name][k,i] = rank_percent

In [16]:
total_areas = dict()
for j in range(len(filters[:-1])):
    Keyname = 'Exgalm5_color_with_cuts_AGN_{0}{1}_mean'.format(filters[j], filters[j+1])
    for Key_aux in Keys:
        if Key_aux[1]==Keyname:
            Key = Key_aux
            break
    color_name = "{0}-{1}".format(filters[j], filters[j+1])
    total_areas[color_name] = get_opsim_areas(Key, bundleDicts)

#### Save the table

In [17]:
for i, z in enumerate(zs):
    fname = "Table_color_excess_z{0}_WFDonly.txt".format(z)
    tab_file = open(fname,"w")
    for k,run in enumerate(list(bundleDicts.keys())):
        tab_file.write("{0:50s}".format(run))
        
        for j in range(len(filters[:-1])):
            color_name = "{0}-{1}".format(filters[j], filters[j+1])
            tab_file.write(" {0:10.2f} {1:5.1f} {2:10.0f}".format(mds[color_name][k,i],
                                                            mds_rank[color_name][k,i],
                                                            total_areas[color_name][k]))  

        tab_file.write("\n")
    tab_file.close()